In [2]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 50.2 MB/s eta 0:00:00


In [1]:
import joblib
import torch
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from tqdm.notebook import tqdm
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import QED
from rdkit.Chem import MolSurf
from rdkit.Chem import rdFreeSASA as SASA
import os
#from google.colab import drive
import operator
#drive.mount('/content/drive')
from dataframe_pipeline_script import Dataset
from Toxformer_EncoderDecoder_Classifier import ClassificationTransformer 

In [2]:
#path = "/content/drive/My Drive/Toxicity Project/DATA/fingerprint_df.csv"
path = "DATA/fingerprint_df.csv"
df = pd.read_csv(path)
df.head()

,index,Unnamed: 0.1,Unnamed: 0,id,title,common_name,description,cas,pubchem_id,chemical_formula,...,fp122,fp123,fp124,fp125,fp126,fp127,fp128,binary_rat_ld50,LD50_level,LD50_rat_level
0,0,0,0,2,T3D0001,Arsenic,Arsenic(As) is a ubiquitous metalloid found in...,7440-38-2,104734.0,As,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0
1,1,1,3,5,T3D0004,Vinyl chloride,"Vinyl chloride is a man-made organic compound,...",75-01-4,6338.0,C2H3Cl,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
2,2,2,4,7,T3D0006,Benzene,"Benzene is a toxic, volatile, flammable liquid...",71-43-2,241.0,C6H6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3.0
3,3,3,5,8,T3D0007,Cadmium,Cadmium (group IIB of the periodic table of el...,7440-43-9,31193.0,Cd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
4,4,5,9,13,T3D0012,Clofenotane,Insecticide. Clofenotane is a major component ...,50-29-3,3036.0,C14H9Cl5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
B = 8

In [4]:
testset_torch = Dataset(data=df, label="LD50_rat_level", purpose="test",representation="both", shuffle=True, qed=False, max_len=125)


Test
Length of longest string: 125
ignored: 0
token lengths: 126
finger lengths: 128
Largest value: 124.0
Number of unique embedding values: 20
x shape: torch.Size([6, 254])
t shape: torch.Size([6, 3])


In [5]:
"""
Loading Categorical transformer model
"""
dropout = 0.8
n_embd = 512
n_head = 1
n_layers = 1
transformer = ClassificationTransformer(n_embd=n_embd,n_head=n_head,n_layers=n_layers,seq_len=126,dropout=dropout,n_prop=128,n_class=int(max(df.LD50_rat_level)))
total_params = sum(param.numel() for param in transformer.parameters())
print(f"Total model parameters: {total_params}")
transformer.load_state_dict(torch.load("Saved_models/toxformer_categorical_encoderdecoder_params.pth"))

Total model parameters: 6434819


<All keys matched successfully>

In [8]:
device = torch.device("cpu")
transformer.eval()
transformer = transformer.to(device)
print(testset_torch.make()[0])
y = transformer(testset_torch.make()[0])
print(torch.argmax(testset_torch.make()[1],dim=-1))
print(torch.argmax(y,dim=-1))

tensor([[37., 32., 67.,  ...,  0.,  0.,  0.],
        [37., 32., 79.,  ...,  1.,  0.,  0.],
        [37., 32., 67.,  ...,  1.,  0.,  0.],
        [37., 32., 67.,  ...,  0.,  0.,  0.],
        [37., 32., 79.,  ...,  0.,  0.,  0.],
        [37., 32., 67.,  ...,  1.,  0.,  0.]], dtype=torch.float64)


RuntimeError: expected scalar type Double but found Float

In [23]:
"""
Loading random forest models
"""
suffix = ""
#suffix = "/content/drive/My Drive/Toxicity Project/"
rf_cat = joblib.load(suffix+"Saved_models/RandomForest_Classifier.joblib")
rf_reg = joblib.load(suffix+"Saved_models/RandomForest_Regressor.joblib")

C:\Anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own ris

In [24]:
testset_cat = Dataset(data=df,label="LD50_rat_level", purpose="test", fingerprint=True, shuffle=False, max_len=192, qed=False, tensor=False)
x_test, t_test = testset_cat.make()


Test
ignored: 0
Largest embedding value: 1.0
Number of unique embedding values: 2
x shape: (6, 128)
t shape: (6,)


In [25]:
pred = rf_cat.predict(x_test)
print(pred)
print(t_test)

[0 1 2 0 0 0]
[0. 1. 2. 1. 2. 0.]


In [16]:
testset_reg = Dataset(data=df,label="LD50_rat", purpose="test", fingerprint=True, shuffle=False, max_len=192, qed=False, tensor=False)
x_test, t_test = testset_reg.make()


Test
ignored: 0
Largest embedding value: 1.0
Number of unique embedding values: 2
x shape: (6, 128)
t shape: (6,)


In [177]:
pred = rf_reg.predict(x_test)
print(pred)
print(t_test)

[ 976.45    904.575  3645.465  1029.521   290.2472 1658.6955]
[8.700e+01 6.390e+02 5.628e+03 6.360e+02 9.000e+04 2.630e-01]


In [27]:
level_dict = {0:"High", 1:"Medium", 2:"Low"}

In [58]:
molecule = "FP(=O)(OC(C)C)C"
transformer.eval()
logit = transformer(testset_torch.tokenize(molecule, max_len=183))
p = torch.nn.functional.softmax(logit)
pred = torch.argmax(logit).item()
print(f"\nTransformer prediction: {level_dict[pred]} (Confidence: {max(p)*100:0.1f}%)")
pred_rf_cat = rf_cat.predict(np.expand_dims(testset_cat.printfinger(molecule), axis=0))
print(f"Random forest classifier prediction: {level_dict[pred_rf_cat[0]]}")
pred_rf_reg = rf_reg.predict(np.expand_dims(testset_reg.printfinger(molecule), axis=0))
print(f"Random forest regressor prediction: {pred_rf_reg[0]} mg/kg")


Transformer prediction: Low (Confidence: 56.7%)
Random forest classifier prediction: High
Random forest regressor prediction: 751.184 mg/kg


C:\Users\Pietro Willi\AppData\Local\Temp\ipykernel_34376\3614215548.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p = torch.nn.functional.softmax(logit)
